In [2]:
import pandas as pd
baseline = []
images_list = [598,1048,1198,1498,1798]
last = 0
for u, i in enumerate(images_list):
    baseline.extend([u]*(i-last))
    last = i
baseline = [0]+baseline

df = pd.read_csv('data/sample_submission.csv')
df.label = baseline
df.to_csv('data/labels.csv',index = False)

In [10]:
import model
import dataset
import torch
import albumentations as A
import pytorch_lightning as pl
from pytorch_lightning import Trainer

if __name__ == '__main__':
    trainer = Trainer(max_epochs = 30,gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(256,256,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':32,'train_tr':train_tr,'val_tr':val_tr})
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)
    output = torch.tensor([])

    for i in range(len(out)):
        output = torch.cat((output,torch.tensor(out[i][0]).argmax(1)))
    output = output.int().tolist()
    dataset.writeSub(output)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b3


Predicting: 0it [00:00, ?it/s]